# Russian Synodal Bible (1885)

## Import

### Libraries

In [31]:
import os
from lxml import etree
from bs4 import BeautifulSoup

from backend import *

In [32]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

### Definitions

In [33]:
bibleTXT = './texts/bible/sinodalnyi-perevod.txt' 
booksJSON = './texts/bible/booksDict.json'
bibleJSON = './texts/bible/bible.json'
bibleXML = './texts/bible/bible.xml'

bibleOHCO = ['test', 'book', 'chap', 'verse']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

chap_lines_re = '^===\s(\d{1,3})\s===$'
book_lines_re = '^==\s(.+)\s==$'
matt_name = 'От Матфея святое благовествование'

In [34]:
with open(booksJSON) as json_file: 
    booksDict = json.load(json_file)

## Pre-Processing

### Import Text into DF

### Tidy Up

### Find Parts

### Assign OHCO Labels

### Swap Full Book Name for Abbreviation

### Export to JSON

## Processing

In [5]:
BibDf = pd.read_json(bibleJSON, orient='index')
BibDf.index.name = 'v_id'
BibTextDf = BibDf[['text']]
BibLibDf = BibDf[bibleOHCO]
BibDf

,test,book,chap,verse,text
v_id,,,,,
1,O,Gen,1,1,В начале сотворил Бог небо и землю.
2,O,Gen,1,2,"Земля же была безвидна и пуста, и тьма над без..."
3,O,Gen,1,3,И сказал Бог: да будет свет. И стал свет.
4,O,Gen,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
5,O,Gen,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
37106,N,Rev,22,17,И Дух и невеста говорят: прииди! И слышавший д...
37107,N,Rev,22,18,И я также свидетельствую всякому слышащему сло...
37108,N,Rev,22,19,и если кто отнимет что от слов книги пророчест...


### Make XML

In [ ]:
root = etree.Element("bible")
print(root)
print(root.tag)
#bibleTree = etree.ElementTree()

In [ ]:
for v in BibDf.index.to_list(): 
    

In [ ]:
root.append(etree.Element("testament"))

In [ ]:
root.append(etree.Element(""))

In [ ]:
etree.tostring(root, pretty_print=True)

## OHCO DFs

In [13]:
#testsDict = dict(enumerate(BibDf.test.unique()))
testsDict = dict([(value, key) for key, value in dict(enumerate(BibDf.test.unique())).items()])
booksDict = dict([(value, key) for key, value in dict(enumerate(BibDf.book.unique())).items()])

### Testaments

In [11]:
TestsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:1]).text], columns=['test', 'text']).set_index(np.arange(1,len(BibDf.test.unique())+1))
TestsDf.index.name = 't_id'
#TestsDf 

### Books

In [14]:
BooksDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:2]).text], columns=[('test', 'book'), 'text'])
BooksDf[['test','book']] = pd.DataFrame(list(BooksDf[('test', 'book')]), index=BooksDf.index, columns=bibleOHCO[:2])
del BooksDf[('test', 'book')]
BooksDf = BooksDf.replace({"test": testsDict, "book": booksDict}).sort_values(by=bibleOHCO[:2], ascending=[True, True])
BooksDf = BooksDf.reset_index().drop(['index'], axis=1).set_index(np.arange(1,len(BibDf.book.unique())+1))
BooksDf.index.name = 'b_id'
BooksDf = BooksDf.replace({"test":dict(enumerate(BibDf.test.unique())), "book":dict(enumerate(BibDf.book.unique()))})
#BooksDf = BooksDf.reset_index().set_index(bibleOHCO[:2])
#BooksDf 

### Chapters

In [17]:
ChapsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in BibDf.groupby(bibleOHCO[:3]).text], columns=[('test', 'book', 'chap'), 'text'])
ChapsDf[['test','book','chap']] = pd.DataFrame(list(ChapsDf[('test', 'book', 'chap')]), index=ChapsDf.index, columns=bibleOHCO[:3])
del ChapsDf[('test', 'book', 'chap')]
ChapsDf = ChapsDf.replace({"test": testsDict, "book": booksDict}).sort_values(by=bibleOHCO[:3], ascending=[True, True, True])
ChapsDf = ChapsDf.reset_index().drop(['index'], axis=1).set_index(np.arange(1,len(ChapsDf)+1))
ChapsDf.index.name = 'c_id'
ChapsDf = ChapsDf.replace({"test":dict(enumerate(BibDf.test.unique())), "book":dict(enumerate(BibDf.book.unique()))})
#ChapsDf = ChapsDf.reset_index().set_index(bibleOHCO[:3])
ChapsDf

,text,test,book,chap
c_id,,,,
1,В начале сотворил Бог небо и землю. Земля же б...,O,Gen,1
2,Так совершены небо и земля и все воинство их. ...,O,Gen,2
3,"Змей был хитрее всех зверей полевых, которых с...",O,Gen,3
4,"Адам познал Еву, жену свою; и она зачала, и ро...",O,Gen,4
5,Вот родословие Адама: когда Бог сотворил челов...,O,Gen,5
...,...,...,...,...
1357,"После сего я увидел иного Ангела, сходящего с ...",N,Rev,18
1358,После сего я услышал на небе громкий голос как...,N,Rev,19
1359,"И увидел я Ангела, сходящего с неба, который и...",N,Rev,20


In [20]:
%%time
ChapsTokenDf = nat_parse(ChapsDf)

CPU times: user 49min 24s, sys: 48min 25s, total: 1h 37min 50s
Wall time: 17min 28s


In [72]:
%%time
VerseTokenDf = nat_parse(BibTextDf)

CPU times: user 7h 56min 11s, sys: 11h 17min 13s, total: 19h 13min 24s
Wall time: 1h 26min 10s


In [75]:
VerseTokenDf

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,aspect,case,degree,gender,mood,number,person,tense,verb_form,voice
0,1,0,1,В,1_1,1_2,case,ADP,в,None,None,None,None,None,None,None,None,None,None,None
1,1,2,8,начале,1_2,1_3,obl,NOUN,начало,Inan,None,Loc,None,Neut,None,Sing,None,None,None,None
2,1,9,17,сотворил,1_3,1_0,root,VERB,сотворить,None,Perf,None,None,Masc,Ind,Sing,None,Past,Fin,Act
3,1,18,21,Бог,1_4,1_3,nsubj,PROPN,бог,Anim,None,Nom,None,Masc,None,Sing,None,None,None,None
4,1,22,26,небо,1_5,1_3,obj,NOUN,небо,Inan,None,Acc,None,Neut,None,Sing,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,37110,32,38,Христа,1_5,1_4,flat:name,PROPN,христос,Anim,None,Gen,None,Masc,None,Sing,None,None,None,None
5,37110,39,41,со,1_6,1_8,case,ADP,с,None,None,None,None,None,None,None,None,None,None,None
6,37110,42,47,всеми,1_7,1_8,det,DET,весь,None,None,Ins,None,None,None,Plur,None,None,None,None
7,37110,48,52,вами,1_8,1_4,nmod,PRON,вы,None,None,Ins,None,None,None,Plur,2,None,None,None


In [76]:
VerseTokenDf.to_pickle('./proc/BibleVerseTokenDf.pkl')

In [49]:
tokenDf = pd.read_pickle('./proc/BibleTokenDf.pkl')
tokenDf.set_index(['v_id', 'token_id'])

token_num start  stop      text head_id        rel    pos  \
v_id token_id                                                             
1    1_1              0     0     1         В     1_2       case    ADP   
     1_2              1     2     8    начале     1_3        obl   NOUN   
     1_3              2     9    17  сотворил     1_0       root   VERB   
     1_4              3    18    21       Бог     1_3      nsubj  PROPN   
     1_5              4    22    26      небо     1_3        obj   NOUN   
...                 ...   ...   ...       ...     ...        ...    ...   
1361 26_5             4  2257  2263    Христа    26_4  flat:name  PROPN   
     26_6             5  2264  2266        со    26_8       case    ADP   
     26_7             6  2267  2272     всеми    26_8        det    DET   
     26_8             7  2273  2277      вами    26_4       nmod   PRON   
     27_1             0  2279  2284     Аминь    27_0       root  PROPN   

                   lemma  anim aspect aspect  case degree gender  mood number  \
v_id token_id                                                                   
1    1_1               в  None   None   None  None   None   None  None   None   
     1_2          начало  Inan   None   None   Loc   None   Neut  None   Sing   
     1_3       сотворить  None   Perf   Perf  None   None   Masc   Ind   Sing   
     1_4             бог  Anim   None   None   Nom   None   Masc  None   Sing   
     1_5            небо  Inan   None   None   Acc   None   Neut  None   Sing   
...                  ...   ...    ...    ...   ...    ...    ...   ...    ...   
1361 26_5        христос  Anim   None   None   Gen   None   Masc  None   Sing   
     26_6              с  None   None   None  None   None   None  None   None   
     26_7           весь  None   None   None   Ins   None   None  None   Plur   
     26_8             вы  None   None   None   Ins   None   None  None   Plur   
     27_1          аминь  Inan   None   None   Gen   None    Fem  None   Sing   

              person tense verb_form voice  
v_id token_id                               
1    1_1        None  None      None  None  
     1_2        None  None      None  None  
     1_3        None  Past       Fin   Act  
     1_4        None  None      None  None  
     1_5        None  None      None  None  
...              ...   ...       ...   ...  
1361 26_5       None  None      None  None  
     26_6       None  None      None  None  
     26_7       None  None      None  None  
     26_8          2  None      None  None  
     27_1       None  None      None  None  

[676137 rows x 20 columns]

In [63]:
tokenDf.query("pos == 'PRON'").lemma.unique()

array(['он', 'она', 'который', 'это', 'они', 'я', 'вы', 'весь', 'что',
       'ты', 'мы', 'оно', 'себя', 'кто', 'тот', 'никто', 'друг', 'дать',
       'ничто', 'дам', 'твой', 'мою', 'какой', 'мой', 'некто', 'народ',
       'уста', 'бог', 'пред', 'твою', 'нечто', 'видеть', 'жертву', 'вот',
       'ваш', 'наш', 'господь', 'того', 'свой', 'видите', 'подойти',
       'вспомните', 'божий', 'всякое', 'скажет', 'сие', 'уловить', 'твое',
       'свою', 'изобличить', 'лето', 'откровение', 'слезами', 'мое',
       'веревка', 'издревле', 'благословенный', 'прочее', 'всевышний',
       'такой', 'помните', 'многое', 'свое', 'господин', 'обидел',
       'обыкновение', 'желать'], dtype=object)

In [52]:
ChapsTokenDf.set_index(['p_id', 'token_id'])

start  stop      text head_id        rel    pos      lemma  \
p_id token_id                                                              
1    1_1          0     1         В     1_2       case    ADP          в   
     1_2          2     8    начале     1_3        obl   NOUN     начало   
     1_3          9    17  сотворил     1_0       root   VERB  сотворить   
     1_4         18    21       Бог     1_3      nsubj  PROPN        бог   
     1_5         22    26      небо     1_3        obj   NOUN       небо   
...             ...   ...       ...     ...        ...    ...        ...   
1361 26_5      2257  2263    Христа    26_4  flat:name  PROPN    христос   
     26_6      2264  2266        со    26_8       case    ADP          с   
     26_7      2267  2272     всеми    26_8        det    DET       весь   
     26_8      2273  2277      вами    26_4       nmod   PRON         вы   
     27_1      2279  2284     Аминь    27_0       root  PROPN      аминь   

               anim aspect  case degree gender  mood number person tense  \
p_id token_id                                                              
1    1_1       None   None  None   None   None  None   None   None  None   
     1_2       Inan   None   Loc   None   Neut  None   Sing   None  None   
     1_3       None   Perf  None   None   Masc   Ind   Sing   None  Past   
     1_4       Anim   None   Nom   None   Masc  None   Sing   None  None   
     1_5       Inan   None   Acc   None   Neut  None   Sing   None  None   
...             ...    ...   ...    ...    ...   ...    ...    ...   ...   
1361 26_5      Anim   None   Gen   None   Masc  None   Sing   None  None   
     26_6      None   None  None   None   None  None   None   None  None   
     26_7      None   None   Ins   None   None  None   Plur   None  None   
     26_8      None   None   Ins   None   None  None   Plur      2  None   
     27_1      Inan   None   Gen   None    Fem  None   Sing   None  None   

              verb_form voice  
p_id token_id                  
1    1_1           None  None  
     1_2           None  None  
     1_3            Fin   Act  
     1_4           None  None  
     1_5           None  None  
...                 ...   ...  
1361 26_5          None  None  
     26_6          None  None  
     26_7          None  None  
     26_8          None  None  
     27_1          None  None  

[676137 rows x 18 columns]

In [64]:
ChapsTokenDf.query("pos == 'PRON'").lemma.unique()

array(['он', 'она', 'который', 'это', 'они', 'я', 'вы', 'весь', 'что',
       'ты', 'мы', 'оно', 'себя', 'кто', 'тот', 'никто', 'друг', 'дать',
       'ничто', 'дам', 'твой', 'мою', 'какой', 'мой', 'некто', 'народ',
       'уста', 'бог', 'пред', 'твою', 'нечто', 'видеть', 'жертву', 'вот',
       'ваш', 'наш', 'господь', 'того', 'свой', 'видите', 'подойти',
       'вспомните', 'божий', 'всякое', 'скажет', 'сие', 'уловить', 'твое',
       'свою', 'изобличить', 'лето', 'откровение', 'слезами', 'мое',
       'веревка', 'издревле', 'благословенный', 'прочее', 'всевышний',
       'такой', 'помните', 'многое', 'свое', 'господин', 'обидел',
       'обыкновение', 'желать'], dtype=object)

In [ ]:
%%time
TokenDf = nat_parse(BibTextDf).set_index(['p_id', 'token_num'])

In [ ]:
TokenDf.to_pickle('./proc/BibleTokenDf.pkl')